In [ ]:
import os
from train_ner import *
from pathlib import Path

In [6]:
__file__ = "__init__.py"

base_dir = Path(__file__).resolve().parents[1]

# data_dir = os.path.join(base_dir, "data")

paths = {

    "models": os.path.join(base_dir, "models"),

}


In [7]:
model = HybridModel(categorical_columns, numerical_columns, embedding_dim=8)
if "laptop_evaluator.pth" not in os.listdir(paths["models"]):
    print("Model not found. Training a new model...")
    model_train()

model.load_state_dict(
    torch.load(
        os.path.join(paths["models"], "laptop_evaluator.pth"), weights_only=False
    )
)
model.eval()

def Searching(query, top_k=5):
    filtered = df[df["DEVICE"].str.contains(query, case=False)].copy()
    
    if filtered.empty:
        return filtered
    numerical_input = torch.tensor(filtered[numerical_columns].values, dtype=torch.float)
    categorical_input = torch.tensor(filtered[categorical_columns].values, dtype=torch.long)

    model.eval()
    with torch.no_grad():
        outputs = model(numerical_input, categorical_input)
        predicted_scores = outputs.squeeze().numpy()

    filtered["PREDICTED_SCORE"] = predicted_scores
    sorted_filtered = filtered.sort_values(
        by=["PREDICTED_SCORE", "CPU PASSMARK RESULT", "GPU PASSMARK (G3D) RESULT"],
        ascending=[False, False, False]
    ).head(top_k)

    return sorted_filtered

In [8]:
recommendations = Searching("RTX 3060", top_k=10)
for idx, laptop in recommendations.iterrows():
    print(f"{laptop['DEVICE']} - Score: {laptop['SCORE']:.1f}")

Asus ROG Strix Scar 17 (2022) G733 17.3 Inches Intel Core i9-12900H 2.5GHz / Nvidia GeForce RTX 3060 Laptop / 64GB RAM / 512GB SSD - Score: -0.5
Asus ROG Zephyrus M16 (2022) GU603 16 Inches Intel Core i9-12900H 2.5GHz / Nvidia GeForce RTX 3060 Laptop / 8GB RAM / 512GB SSD - Score: -0.5
Asus Zenbook Pro 16X OLED (UX7602) 16 Inches Intel Core i9-12900H 2.5GHz / Nvidia GeForce RTX 3060 Laptop / 32GB RAM / 512GB SSD - Score: -0.5
Asus ROG Strix Scar 15 (2022) G533 15.6 Inches Intel Core i9-12900H 2.5GHz / Nvidia GeForce RTX 3060 Laptop / 64GB RAM / 512GB SSD - Score: -0.5
Dell Alienware m15 R7 (2022) 15.6 Inches QHD Intel Core i7-12700H 2.3GHz / Nvidia GeForce RTX 3060 Laptop / 64GB RAM / 512GB SSD - Score: -0.6
Dell Alienware x17 R2 17.3 Inches FHD Intel Core i7-12700H / Nvidia GeForce RTX 3060 Laptop / 64GB RAM / 512GB SSD - Score: -0.6
MSI GP66 Vector 12UE 272 15.6 Inches FHD Intel Core i7-12700H 2.3GHz / Nvidia GeForce RTX 3060 Laptop / 16GB RAM / 512GB SSD - Score: -0.6
Dell Alienware